This notebook leverages openai language capability to read through papers of waabi and give me a summary on Prof. Urtasan's critique on other end 2 end apporach. 
Note: The notebook inherited from scrap-n-send.ipython which is to download all pdf files from the arxiv and then send to gpt4 to extract the content. Once content is extracted, it will be used to update the existing row in the table.

In [2]:
# 
import pandas as pd
import openpyxl



In [3]:
# necessary libraries for scraping
%pip install requests
%pip install beautifulsoup4



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Next, we are going to extrace the text from one pdf file and send to gpt4 to extract the content. 

In [10]:


# obtain all path of all pdf files in the /Users/htan/Library/Mobile Documents/iCloud~md~obsidian/Documents folder
import os

# Define the directory path
directory_path = '/Users/htan/Library/Mobile Documents/iCloud~md~obsidian/Documents/WaabiResearch'

# Retrieve all PDF file paths
pdf_files = [os.path.join(directory_path, f) for f in os.listdir(directory_path) if f.endswith('.pdf')]

print(pdf_files)


['/Users/htan/Library/Mobile Documents/iCloud~md~obsidian/Documents/WaabiResearch/Learning Realistic Traffic Agents in Closed-loop.pdf', '/Users/htan/Library/Mobile Documents/iCloud~md~obsidian/Documents/WaabiResearch/TRAVL-main-1.pdf', '/Users/htan/Library/Mobile Documents/iCloud~md~obsidian/Documents/WaabiResearch/selfplay_paper.pdf', '/Users/htan/Library/Mobile Documents/iCloud~md~obsidian/Documents/WaabiResearch/lidar-dg-paper.pdf', '/Users/htan/Library/Mobile Documents/iCloud~md~obsidian/Documents/WaabiResearch/Towards Scalable Coverage-Based Testing of Autonomous Vehicles.pdf', '/Users/htan/Library/Mobile Documents/iCloud~md~obsidian/Documents/WaabiResearch/G3R Gradient Guided Generalizable Reconstruction.pdf', '/Users/htan/Library/Mobile Documents/iCloud~md~obsidian/Documents/WaabiResearch/GoRela-paper-2.pdf', '/Users/htan/Library/Mobile Documents/iCloud~md~obsidian/Documents/WaabiResearch/UnO Unsupervised Occupancy Fields for Perception and Forecasting.pdf', '/Users/htan/Librar

# Checking point: save df from the excel and intermediate variables for df operation.

In [11]:
# I'm using workspace to save the df and other intermediate variables. 
# save all variables in the global scope to a pickle file
import pickle
import types  # Import the types module to check for specific types
import sys

# Create a dictionary of serializable variables
variables_to_save = {
    name: value for name, value in globals().items()
    if not name.startswith("__") and not callable(value) and not isinstance(value, (types.ModuleType, types.FunctionType, types.BuiltinFunctionType))
}

# Save to a file
with open('variables.pkl', 'wb') as f:
    pickle.dump(variables_to_save, f)

print("Variables saved successfully.")

Variables saved successfully.


# Checkpoint: load the variables from the file

In [12]:
# load the variables from the file
import pickle
with open('variables.pkl', 'rb') as f:
    loaded_variables = pickle.load(f)

# Restore variables to the global scope
globals().update(loaded_variables)

In [18]:
from openai import OpenAI
# Load the PDF file
from PyPDF2 import PdfReader

client = OpenAI()

# this code is to extract text from the local pdf file and then send to gpt4 together with the prompt

# Extract text from the PDF
def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

text = extract_text_from_pdf(pdf_files[0])

# measure the length of text
print('length of text:', len(text))

# since the input size needs to be less than 128k tokens, we need to process one page at a time
prompt = f"step 1: reading through the paper {text}, specify the title of the paper, and summarize the arthuors opinion on the end 2 end approach, including the pros and cons in the specific use cases. ; \
step 2, summarize the problem statement of the end 2 end approach if any, and the solution proposed by the authors ; if there is no problem statement releveent to the end 2 end approach, just say so; \
step 3, for each the statement in step 1 and step 2, give the quote from the paper." 

# Send the prompt to the OpenAI API and get the response
completion = client.chat.completions.create(
    model="gpt-4o-mini",  # Using "gpt-4" or another appropriate model
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", 
         "content": prompt
        }
    ]
)

# Print the extracted content
print(completion.choices[0].message)

length of text: 65893
ChatCompletionMessage(content='### Step 1: Title and Author\'s Opinion on End-to-End Approach\n\n**Title of the Paper:** Learning Realistic Traffic Agents in Closed-loop\n\n**Author\'s Opinion on End-to-End Approach:**\nThe authors highlight that traditional approaches used in the self-driving industry often lack the realism necessary for effective traffic simulation. They discuss the pros and cons of the end-to-end approach, particularly how imitation learning (IL) can capture human-like driving behavior but suffers from fundamental limitations, including unrealistic behaviors due to a lack of explicit safety knowledge. \n\n**Pros:**\n- IL provides a framework for learning human-like driving policies by imitating expert demonstrations.\n- Closed-loop IL allows the model to adapt to states induced by its own actions, leading to improved robustness against distribution shifts.\n\n**Cons:**\n- Pure IL can lead to high rates of traffic infractions, such as collisions

In [19]:
# print the completion.choices[0].message.content in a readable format
print(completion.choices[0].message.content)

### Step 1: Title and Author's Opinion on End-to-End Approach

**Title of the Paper:** Learning Realistic Traffic Agents in Closed-loop

**Author's Opinion on End-to-End Approach:**
The authors highlight that traditional approaches used in the self-driving industry often lack the realism necessary for effective traffic simulation. They discuss the pros and cons of the end-to-end approach, particularly how imitation learning (IL) can capture human-like driving behavior but suffers from fundamental limitations, including unrealistic behaviors due to a lack of explicit safety knowledge. 

**Pros:**
- IL provides a framework for learning human-like driving policies by imitating expert demonstrations.
- Closed-loop IL allows the model to adapt to states induced by its own actions, leading to improved robustness against distribution shifts.

**Cons:**
- Pure IL can lead to high rates of traffic infractions, such as collisions and off-road driving.
- There are challenges in capturing the nuan

In [21]:
# process the second pdf file
text = extract_text_from_pdf(pdf_files[1])

# measure the length of text
print('length of text:', len(text))

# since the input size needs to be less than 128k tokens, we need to process one page at a time
prompt = f"step 1: reading through the paper {text}, specify the title of the paper, and summarize the arthuors opinion on the end 2 end approach, including the pros and cons in the specific use cases. ; \
step 2, summarize the problem statement of the end 2 end approach if any, and the solution proposed by the authors ; if there is no problem statement releveent to the end 2 end approach, just say so; \
step 3, for each the statement in step 1 and step 2, give the quote from the paper." 

# Send the prompt to the OpenAI API and get the response
completion = client.chat.completions.create(
    model="gpt-4o-mini",  # Using "gpt-4" or another appropriate model
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", 
         "content": prompt
        }
    ]
)

# print the completion.choices[0].message.content in a readable format
print(completion.choices[0].message.content)

length of text: 74261
### Step 1: Title and Summary of Authors' Opinion on End-to-End Approach

**Title of the Paper:** "Rethinking Closed-loop Training for Autonomous Driving"

**Summary of Authors' Opinion on the End-to-End Approach:**
The authors recognize end-to-end approaches like behavior cloning as popular methods in self-driving vehicle development. However, they critique such methods for their reliance on open-loop training, which can lead to distribution shift and insufficient handling of the complexities of real-world driving. They argue that while end-to-end methods, including behavior cloning, can efficiently leverage large amounts of driving data, they are generally not adequate for ensuring safe and effective decision-making in autonomous systems, especially in complex traffic scenarios. They outline the following notable pros and cons of end-to-end approaches:

- **Pros:**
  - End-to-end systems can leverage extensive driving data and mimic human driving behaviors effec

In [22]:
# save the completion.choices[0].message.content to the local varilalbe so that it can be used later    
digest_content = completion.choices[0].message.content

# process the third pdf file
text = extract_text_from_pdf(pdf_files[2])

# measure the length of text
print('length of text:', len(text))

# since the input size needs to be less than 128k tokens, we need to process one page at a time
prompt = f"step 1: reading through the paper {text}, specify the title of the paper, and summarize the arthuors opinion on the end 2 end approach, including the pros and cons in the specific use cases. ; \
step 2, summarize the problem statement of the end 2 end approach if any, and the solution proposed by the authors ; if there is no problem statement releveent to the end 2 end approach, just say so; \
step 3, for each the statement in step 1 and step 2, give the quote from the paper." 

# Send the prompt to the OpenAI API and get the response
completion = client.chat.completions.create(
    model="gpt-4o-mini",  # Using "gpt-4" or another appropriate model
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", 
         "content": prompt
        }
    ]
)

# print the completion.choices[0].message.content in a readable format
print(completion.choices[0].message.content)


FloatObject (b'0.000000000000-5684342') invalid; use 0.0 instead
FloatObject (b'0.000000000000-5684342') invalid; use 0.0 instead
FloatObject (b'0.000000000000-5684342') invalid; use 0.0 instead
FloatObject (b'0.000000000000-5684342') invalid; use 0.0 instead


length of text: 78944
### Step 1: Summary of Authors' Opinion on End-to-End Approach

The title of the paper is "Learning to Drive via Asymmetric Self-Play" by Chris Zhang et al.

**Authors' Opinion:**
The authors express a cautious perspective on the end-to-end (E2E) approach for training autonomous driving systems. Their view acknowledges the potential benefits of an E2E strategy, particularly in generating realistic driving scenarios and improving performance in safety-critical situations. However, they highlight significant limitations, especially regarding generalization to edge cases.

**Pros of End-to-End Approach:**
- The E2E approach can effectively leverage training data to fine-tune policies that perform complex driving tasks, especially when trained on diverse scenarios.
- E2E methods benefit from the real-world data representation that can help in scenarios mimicking actual driving behaviors.

**Cons of End-to-End Approach:**
- End-to-end models often struggle with general

In [24]:
# attached the completion.choices[0].message.content to the digest_content  
digest_content = digest_content + completion.choices[0].message.content



In [25]:
# print out the digest_content
print(digest_content)

### Step 1: Title and Summary of Authors' Opinion on End-to-End Approach

**Title of the Paper:** "Rethinking Closed-loop Training for Autonomous Driving"

**Summary of Authors' Opinion on the End-to-End Approach:**
The authors recognize end-to-end approaches like behavior cloning as popular methods in self-driving vehicle development. However, they critique such methods for their reliance on open-loop training, which can lead to distribution shift and insufficient handling of the complexities of real-world driving. They argue that while end-to-end methods, including behavior cloning, can efficiently leverage large amounts of driving data, they are generally not adequate for ensuring safe and effective decision-making in autonomous systems, especially in complex traffic scenarios. They outline the following notable pros and cons of end-to-end approaches:

- **Pros:**
  - End-to-end systems can leverage extensive driving data and mimic human driving behaviors effectively. 
  - They can 

In [26]:
# process the fourth pdf file
text = extract_text_from_pdf(pdf_files[3])

# measure the length of text
print('length of text:', len(text))

# since the input size needs to be less than 128k tokens, we need to process one page at a time
prompt = f"step 1: reading through the paper {text}, specify the title of the paper, and summarize the arthuors opinion on the end 2 end approach, including the pros and cons in the specific use cases. ; \
step 2, summarize the problem statement of the end 2 end approach if any, and the solution proposed by the authors ; if there is no problem statement releveent to the end 2 end approach, just say so; \
step 3, for each the statement in step 1 and step 2, give the quote from the paper." 

# Send the prompt to the OpenAI API and get the response
completion = client.chat.completions.create(
    model="gpt-4o-mini",  # Using "gpt-4" or another appropriate model
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", 
         "content": prompt
        }
    ]
)

# print the completion.choices[0].message.content in a readable format
print(completion.choices[0].message.content)

# attached the completion.choices[0].message.content to the digest_content      
digest_content = digest_content + completion.choices[0].message.content


length of text: 54208
### Step 1: Summary of Authors' Opinion on the End-to-End Approach

**Title of the Paper:** Towards Zero Domain Gap: A Comprehensive Study of Realistic LiDAR Simulation for Autonomy Testing

**Authors' Opinion on the End-to-End Approach:**
The authors highlight the importance of accurately simulating sensor inputs, particularly LiDAR, for autonomous vehicle testing. They emphasize that an end-to-end approach addresses the critical need to ensure that the performance of an autonomy system in simulation matches that in the real world. 

**Pros:**
- **Safety and Scalability:** Testing full autonomy systems in simulation is portrayed as the safest and most scalable method for evaluating autonomous vehicle performance prior to deployment.
- **Matching Performance:** Achieving low domain gap ensures that the simulated environment behaves similarly to real-world conditions, which is vital for trust in the autonomy system.

**Cons:**
- **Challenges in Simulation:** Despit

In [27]:
# process the rest of the pdf files
for pdf_file in pdf_files[4:]:
    text = extract_text_from_pdf(pdf_file)

    # measure the length of text
    print('length of text:', len(text))

    # since the input size needs to be less than 128k tokens, we need to process one page at a time
    prompt = f"step 1: reading through the paper {text}, specify the title of the paper, and summarize the arthuors opinion on the end 2 end approach, including the pros and cons in the specific use cases. ; \
    step 2, summarize the problem statement of the end 2 end approach if any, and the solution proposed by the authors ; if there is no problem statement releveent to the end 2 end approach, just say so; \
    step 3, for each the statement in step 1 and step 2, give the quote from the paper." 

    # Send the prompt to the OpenAI API and get the response
    completion = client.chat.completions.create(
        model="gpt-4o-mini",  # Using "gpt-4" or another appropriate model
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", 
            "content": prompt
            }
        ]
    )

    # print the completion.choices[0].message.content in a readable format
    print(completion.choices[0].message.content)

    # attached the completion.choices[0].message.content to the digest_content      
    digest_content = digest_content + completion.choices[0].message.content

length of text: 46055
### Step 1: Title of the Paper and Authors' Opinion on End-to-End Approach

**Title of the Paper**: Towards Scalable Coverage-Based Testing of Autonomous Vehicles

**Authors' Opinion on the End-to-End Approach**:
The authors analyze various testing methodologies for Autonomous Vehicles (AVs), highlighting that traditional end-to-end approaches can struggle with coverage and accuracy due to the complexities of continuous parameter spaces. While the end-to-end testing approach can provide insights across many scenarios, it is often hindered by the need to discretize parameter spaces, resulting in inaccuracies and inefficiencies when scaling to high dimensions.

**Pros of the End-to-End Approach**:
- It allows for the testing of AV systems in a variety of realistic and complex scenarios.
- It is useful for exploring diverse contextual variations in driving behavior.

**Cons of the End-to-End Approach**:
- It relies on discretizing continuous parameters, which leads t

In [28]:
# print out the digest_content  
print(digest_content)

# save the digest_content to a local file

### Step 1: Title and Summary of Authors' Opinion on End-to-End Approach

**Title of the Paper:** "Rethinking Closed-loop Training for Autonomous Driving"

**Summary of Authors' Opinion on the End-to-End Approach:**
The authors recognize end-to-end approaches like behavior cloning as popular methods in self-driving vehicle development. However, they critique such methods for their reliance on open-loop training, which can lead to distribution shift and insufficient handling of the complexities of real-world driving. They argue that while end-to-end methods, including behavior cloning, can efficiently leverage large amounts of driving data, they are generally not adequate for ensuring safe and effective decision-making in autonomous systems, especially in complex traffic scenarios. They outline the following notable pros and cons of end-to-end approaches:

- **Pros:**
  - End-to-end systems can leverage extensive driving data and mimic human driving behaviors effectively. 
  - They can 

In [29]:
# Not surprisingly that the query doesn't reach me specific details on the explainnability and reasoning of the end 2 end approach. 
# Update the prompt above to ask for the explainnability and reasoning of the end 2 end approach. 

digest_content_explainabilityandreasoning = ''

# process files in the pdf files
for pdf_file in pdf_files:
    text = extract_text_from_pdf(pdf_file)

    # measure the length of text
    print('length of text:', len(text))

    # since the input size needs to be less than 128k tokens, we need to process one page at a time
    prompt = f"step 1: reading through the paper {text}, specify the title of the paper, and summarize the arthuors opinion on the end 2 end approach, including the pros and cons in the specific use cases. ; \
    step 2, given the context of step 1, summarize the opinion or statement on explainnability and reasoning of the end 2 end approach if any. If it's not mentioned, just say so; \
    step 3, for each the statement in step 1 and step 2, give the quote from the paper." 

    # Send the prompt to the OpenAI API and get the response
    completion = client.chat.completions.create(
        model="gpt-4o-mini",  # Using "gpt-4" or another appropriate model
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", 
            "content": prompt
            }
        ]
    )

    # print the completion.choices[0].message.content in a readable format
    print(completion.choices[0].message.content)

    # attached the completion.choices[0].message.content to the digest_content      
    digest_content_explainabilityandreasoning = digest_content_explainabilityandreasoning + completion.choices[0].message.content

# print out the digest_content_explainabilityandreasoning
print(digest_content_explainabilityandreasoning)




length of text: 65893
### Step 1: Title and Author's Opinion on End-to-End Approach

**Title of the Paper:** "Learning Realistic Traffic Agents in Closed-loop"

**Author's Opinion on the End-to-End Approach:**
The authors propose the Reinforcing Traffic Rules (RTR), a hybrid method that combines Imitation Learning (IL) and Reinforcement Learning (RL) to develop realistic traffic agents. They highlight the advantages and disadvantages of traditional approaches, including pure IL and RL techniques, indicating that each alone has shortcomings in capturing realistic human-like behaviors while ensuring safety and compliance with traffic rules. 

**Pros and Cons:**
- **Pros:**
  - The RTR approach leverages both human expert demonstrations (through IL) and safety compliance (through RL), which results in a more balanced and effective learning process. This enables the system to learn from rich, diverse scenarios generated through simulation while maintaining realism in driver behavior. The a

FloatObject (b'0.000000000000-5684342') invalid; use 0.0 instead
FloatObject (b'0.000000000000-5684342') invalid; use 0.0 instead
FloatObject (b'0.000000000000-5684342') invalid; use 0.0 instead
FloatObject (b'0.000000000000-5684342') invalid; use 0.0 instead


length of text: 78944
### Step 1: Summary of Authors' Opinions on End-to-End Approach

**Title of the Paper**: Learning to Drive via Asymmetric Self-Play

**Summary**:
The authors present a critical perspective on traditional end-to-end approaches for driving policy learning. They highlight that while these approaches, such as closed-loop imitation learning and adversarial methods, can achieve certain levels of success, they typically struggle with generating realistic and robust driving policies, especially in complex and interactive scenarios often seen in autonomous driving.

**Pros**:
1. **Efficiency**: The authors acknowledge that end-to-end approaches allow for more direct training pipelines that can quickly assimilate real-world data.
2. **Simplicity**: These models may simplify the complexity of driving by learning from comprehensive datasets through a singular framework.

**Cons**:
1. **Higher Failure Rates**: The authors indicate that end-to-end methods still exhibit higher-t

In [33]:
# save the digest_content_explainabilityandreasoning to a local file
with open('digest_content_explainabilityandreasoning.txt', 'w') as file:
    file.write(digest_content_explainabilityandreasoning)

In [30]:
# Update the prompt above to ask for the interpretability, explainnability and reasoning of the end 2 end approach. 

digest_content_interpretabilityexplainabilityandreasoning = ''

# process files in the pdf files
for pdf_file in pdf_files:
    text = extract_text_from_pdf(pdf_file)

    # measure the length of text
    print('length of text:', len(text))

    # since the input size needs to be less than 128k tokens, we need to process one page at a time
    prompt = f"step 1: reading through the paper {text}, specify the title of the paper, and clarify whether the paper mentions the interpretability, explainnability and reasoning of the end 2 end approach or NOT. If it's not mentioned, just say so; \
    step 2, if the answer on step 1 is yes, then summarize the opinion or statement on interpretability, explainnability and reasoning. otherwise, skip step 2 and step 3; \
    step 3, for each the statement in step 1 and step 2, give the quote from the paper." 

    # Send the prompt to the OpenAI API and get the response
    completion = client.chat.completions.create(
        model="gpt-4o-mini",  # Using "gpt-4" or another appropriate model
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", 
            "content": prompt
            }
        ]
    )

    # print the completion.choices[0].message.content in a readable format
    print(completion.choices[0].message.content)

    # attached the completion.choices[0].message.content to the digest_content      
    digest_content_interpretabilityexplainabilityandreasoning = digest_content_interpretabilityexplainabilityandreasoning + completion.choices[0].message.content

# print out the digest_content_interpretabilityexplainabilityandreasoning
print(digest_content_interpretabilityexplainabilityandreasoning)


length of text: 65893
**Step 1:** The title of the paper is "Learning Realistic Traffic Agents in Closed-loop" by Chris Zhang, James Tu, Lunjun Zhang, Kelvin Wong, Simon Suo, and Raquel Urtasun.

Regarding the interpretability, explainability, and reasoning of the end-to-end approach, the paper does not explicitly mention these concepts. Therefore, my answer is no.

Since step 1 does not mention interpretability, explainability, or reasoning, I will skip step 2 and step 3.
length of text: 74261
### Step 1: Title and Mention of Interpretability, Explainability, and Reasoning

**Title of the Paper:** Rethinking Closed-loop Training for Autonomous Driving

**Mention of Interpretability, Explainability, and Reasoning:** The paper does not explicitly mention interpretability, explainability, or reasoning in the context of an end-to-end approach. 

### Step 2: Skip (As per Step 1 result)

### Step 3: Skip (As per Step 1 result) 

If you need further assistance or information about the paper,

FloatObject (b'0.000000000000-5684342') invalid; use 0.0 instead
FloatObject (b'0.000000000000-5684342') invalid; use 0.0 instead
FloatObject (b'0.000000000000-5684342') invalid; use 0.0 instead
FloatObject (b'0.000000000000-5684342') invalid; use 0.0 instead


length of text: 78944
**Step 1:** The title of the paper is "Learning to Drive via Asymmetric Self-Play." The paper does not mention interpretability, explainability, or reasoning of the end-to-end approach.

**Step 2:** Not applicable, as the answer to Step 1 is no.

**Step 3:** Not applicable, as the answer to Step 1 is no.
length of text: 54208
**Step 1:** The title of the paper is "Towards Zero Domain Gap: A Comprehensive Study of Realistic LiDAR Simulation for Autonomy Testing." The paper does not explicitly mention interpretability, explainability, and reasoning of the end-to-end approach.

**Step 2:** Skipped, as the answer to step 1 is "no."

**Step 3:** Skipped.
length of text: 46055
**Step 1:** The title of the paper is "Towards Scalable Coverage-Based Testing of Autonomous Vehicles." The paper does not mention interpretability, explainability, or reasoning of the end-to-end approach.

**Step 2:** Not applicable, as the answer to Step 1 is no.

**Step 3:** Not applicable, as 

In [32]:
# save the digest_content_interpretabilityexplainabilityandreasoning to a local file
with open('digest_content_interpretabilityexplainabilityandreasoning.txt', 'w') as file:
    file.write(digest_content_interpretabilityexplainabilityandreasoning)


In [27]:
#given completion.choices[0].message, extract the content and show the new rows
#completion.choices[0].message.content
#completion.choices[0].message.content.split('\n')
#completion.choices[0].message.content.split('\n')[-1]
#completion.choices[0].message.content.split('\n')[-1].split(',')
#completion.choices[0].message.content.split('\n')[-1].split(',')[0]
#completion.choices[0].message.content.split('\n')[-1].split(',')[1]
#completion.choices[0].message.content.split('\n')[-1].split(',')[2]

# Extract the table part from the content
content = completion.choices[0].message.content
# Find the start and end of the table
table_start = content.find('| Name')
table_end = content.find('**Summary of Futurewei\'s Support**')
table_markdown = content[table_start:table_end].strip()

# Split the markdown table into rows
rows = table_markdown.split('\n')
# print the table markdown and the shape of table_markdown
print(table_markdown)

# strip away the last two rows since they are not part of the table i'm looking for
rows = rows[:-2]    
#print the rows and shape
print('rows:', rows)
print('shape of rows:', len(rows))


| Name                | Affiliation                                             | Country      | Title of the Paper                                                                             | Futurewei Support                                     |
|---------------------|---------------------------------------------------------|--------------|-----------------------------------------------------------------------------------------------|------------------------------------------------------|
| Mohamed Akrout      | Department of Electrical and Computer Engineering, University of Manitoba | Canada       | Vector Approximate Message Passing with Arbitrary I.I.D. Noise Priors                       | Supported the research through its Discovery Grants Program |
| Tiancheng Gao       | Department of Electrical and Computer Engineering, University of Manitoba | Canada       | Vector Approximate Message Passing with Arbitrary I.I.D. Noise Priors                       | Supported the research

In [30]:
print(table_markdown)
# Show the shape of table_markdown
num_rows = len(table_markdown.split('\n'))  # Count the number of rows
num_columns = len(table_markdown.split('\n')[0].split('|')) - 2  # Count the number of columns (excluding empty first and last elements)

print(f"Shape of table_markdown: ({num_rows}, {num_columns})")

#print the 1st row of table_markdown
print('1st row of table_markdown:', rows[0])
print('1st row of table_markdown:', table_markdown[0:])
#print the 2nd row of table_markdown
print('2nd row of table_markdown:', rows[1])

#print the 3rd row of table_markdown
print('3rd row of table_markdown:', rows[2])

#print the 8th row of table_markdown
print('6th row of table_markdown:', rows[5])

| Name                | Affiliation                                             | Country      | Title of the Paper                                                                             | Futurewei Support                                     |
|---------------------|---------------------------------------------------------|--------------|-----------------------------------------------------------------------------------------------|------------------------------------------------------|
| Mohamed Akrout      | Department of Electrical and Computer Engineering, University of Manitoba | Canada       | Vector Approximate Message Passing with Arbitrary I.I.D. Noise Priors                       | Supported the research through its Discovery Grants Program |
| Tiancheng Gao       | Department of Electrical and Computer Engineering, University of Manitoba | Canada       | Vector Approximate Message Passing with Arbitrary I.I.D. Noise Priors                       | Supported the research

In [ ]:
# the following code is for debugging purpose only 
# Remove rows 52 and after from the DataFrame df 
df = df.iloc[:52]  # Keep rows from the start up to (but not including) row 52
# Extract headers (first row)



In [34]:
# Extract headers (first row)
headers = df.columns.tolist()  # Get headers directly from the DataFrame
print("Headers:", headers)

# Map the headers of table_markdown to the headers of df
# For reach row in the table_markdown, map the values to the new row in dataframe based on headers
for row in rows[2:]:
    new_row = [None] * len(headers)  # Initialize a list with None values
    new_row[headers.index('Author')] = row.split('|')[1].strip()    # Name->Author
    new_row[headers.index('Affiliation')] = row.split('|')[2].strip()   # Affiliation
    new_row[headers.index('国家')] = row.split('|')[3].strip()     # Country->国家
    new_row[headers.index('Title of Publication\\')] = row.split('|')[4].strip()  # Title of Paper
    new_row[headers.index('资助方式')] = row.split('|')[5].strip() # Support
    print("New row:", new_row)
    # append the new row to the df
    df.loc[len(df)] = new_row



Headers: ['Author', 'Affiliation', '国家', 'Title of Publication\\', '来源', '资助方式']
New row: ['Mohamed Akrout', 'Department of Electrical and Computer Engineering, University of Manitoba', 'Canada', 'Vector Approximate Message Passing with Arbitrary I.I.D. Noise Priors', None, 'Supported the research through its Discovery Grants Program']
New row: ['Tiancheng Gao', 'Department of Electrical and Computer Engineering, University of Manitoba', 'Canada', 'Vector Approximate Message Passing with Arbitrary I.I.D. Noise Priors', None, 'Supported the research through its Discovery Grants Program']
New row: ['Faouzi Bellili', 'Department of Electrical and Computer Engineering, University of Manitoba', 'Canada', 'Vector Approximate Message Passing with Arbitrary I.I.D. Noise Priors', None, 'Supported the research through its Discovery Grants Program']
New row: ['Amine Mezghani', 'Department of Electrical and Computer Engineering, University of Manitoba', 'Canada', 'Vector Approximate Message Passin

/var/folders/j3/v1zv7m4j7h71qlqztpqmyh5c0000gp/T/ipykernel_53907/1780599244.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[len(df)] = new_row
/var/folders/j3/v1zv7m4j7h71qlqztpqmyh5c0000gp/T/ipykernel_53907/1780599244.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[len(df)] = new_row
/var/folders/j3/v1zv7m4j7h71qlqztpqmyh5c0000gp/T/ipykernel_53907/1780599244.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  

In [35]:
# since the first pdf file is processed successfully, let's move on to rest of the pdf files
for pdf_file in pdf_files[1:]:
    pdf_path = 'arxiv_pdfs/' + pdf_file
    text = extract_text_from_pdf(pdf_path)
    prompt = f"step 1: extract the info for each author, including name, its affiliation, country, title of the paper, and how does futurewei support the author in the paper {text}; \
    step 2, add a new row for each author in the table;"        

    # Send the prompt to the OpenAI API and get the response
    completion = client.chat.completions.create(
        model="gpt-4o-mini",  # Using "gpt-4" or another appropriate model
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", 
             "content": prompt
            }
        ]
    )
    # Extract the table part from the content
    content = completion.choices[0].message.content
    # Find the start and end of the table
    table_start = content.find('| Name')
    table_end = content.find('**Summary of Futurewei\'s Support**')
    table_markdown = content[table_start:table_end].strip()

    # Split the markdown table into rows
    rows = table_markdown.split('\n')

    # strip away the last two rows since they are not part of the table i'm looking for
    rows = rows[:-2]    

    # For reach row in the table_markdown, map the values to the new row in dataframe based on headers
    for row in rows[2:]:
        new_row = [None] * len(headers)  # Initialize a list with None values
        new_row[headers.index('Author')] = row.split('|')[1].strip()    # Name->Author
        new_row[headers.index('Affiliation')] = row.split('|')[2].strip()   # Affiliation
        new_row[headers.index('国家')] = row.split('|')[3].strip()     # Country->国家
        new_row[headers.index('Title of Publication\\')] = row.split('|')[4].strip()  # Title of Paper
        new_row[headers.index('资助方式')] = row.split('|')[5].strip() # Support
        print("New row:", new_row)
        # append the new row to the df
        df.loc[len(df)] = new_row



    


New row: ['Wei Wu', 'State Key Laboratory of Integrated Services Networks, Xidian University', 'China', 'See SIFT in a Rain', None, 'Supported through research collaboration and funding']
New row: ['Hao Chang', 'State Key Laboratory of Integrated Services Networks, Xidian University', 'China', 'See SIFT in a Rain', None, 'Supported through research collaboration and funding']
New row: ['Zhu Li', 'Department of Computer Science & Electrical Engineering, University of Missouri, Kansas City', 'USA', 'See SIFT in a Rain', None, "Previous position as a senior staff researcher at Futurewei Technology's Media Lab"]


/var/folders/j3/v1zv7m4j7h71qlqztpqmyh5c0000gp/T/ipykernel_53907/3435852039.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[len(df)] = new_row
/var/folders/j3/v1zv7m4j7h71qlqztpqmyh5c0000gp/T/ipykernel_53907/3435852039.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[len(df)] = new_row
/var/folders/j3/v1zv7m4j7h71qlqztpqmyh5c0000gp/T/ipykernel_53907/3435852039.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  

New row: ['Wei Jiang', 'Futurewei Technologies Inc.', 'USA', 'Neural Image Compression Using Masked Sparse Visual Representation', None, 'Futurewei provided the resources and research support for the development of the M-AdaCode method presented in the paper.']
New row: ['Wei Wang', 'Futurewei Technologies Inc.', 'USA', 'Neural Image Compression Using Masked Sparse Visual Representation', None, 'Futurewei offered expertise in neural network design and optimization, facilitating the exploration of SVR-based compression techniques.']
New row: ['Yue Chen', 'Futurewei Technologies Inc.', 'USA', 'Neural Image Compression Using Masked Sparse Visual Representation', None, "Futurewei's infrastructure and technical resources supported the implementation and testing of the proposed image compression methods."]


/var/folders/j3/v1zv7m4j7h71qlqztpqmyh5c0000gp/T/ipykernel_53907/3435852039.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[len(df)] = new_row
/var/folders/j3/v1zv7m4j7h71qlqztpqmyh5c0000gp/T/ipykernel_53907/3435852039.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[len(df)] = new_row
/var/folders/j3/v1zv7m4j7h71qlqztpqmyh5c0000gp/T/ipykernel_53907/3435852039.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  

New row: ['Matt White', 'Linux Foundation', 'San Francisco, CA, USA', 'The Model Openness Framework: Promoting Completeness and Openness for Reproducibility, Transparency, and Usability in Artificial Intelligence', None, 'Futurewei collaborates with the Linux Foundation to promote principles of open science and supports research on AI model transparency and reproducibility.']
New row: ['Ibrahim Haddad', 'Linux Foundation', 'San Francisco, CA, USA', 'The Model Openness Framework: Promoting Completeness and Openness for Reproducibility, Transparency, and Usability in Artificial Intelligence', None, "Futurewei actively engages with the Linux Foundation's initiatives to enhance AI model development practices and improve openness standards in AI research and technology."]
New row: ['Cailean Osborne', 'University of Oxford', 'Oxford, UK', 'The Model Openness Framework: Promoting Completeness and Openness for Reproducibility, Transparency, and Usability in Artificial Intelligence', None, 'Thr

/var/folders/j3/v1zv7m4j7h71qlqztpqmyh5c0000gp/T/ipykernel_53907/3435852039.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[len(df)] = new_row
/var/folders/j3/v1zv7m4j7h71qlqztpqmyh5c0000gp/T/ipykernel_53907/3435852039.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[len(df)] = new_row
/var/folders/j3/v1zv7m4j7h71qlqztpqmyh5c0000gp/T/ipykernel_53907/3435852039.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  

New row: ['Wei Jiang', 'Futurewei Technologies Inc.', 'USA', 'Image and Video Compression using Generative Sparse Representation with Fidelity Controls', None, 'Futurewei provides resources and infrastructure to support the research and development of image and video compression methods.']
New row: ['Wei Wang', 'Futurewei Technologies Inc.', 'USA', 'Image and Video Compression using Generative Sparse Representation with Fidelity Controls', None, 'Futurewei supports the authors by offering technical guidance, collaboration opportunities, and access to relevant datasets.']


IndexError: list index out of range

In [36]:
print(table_markdown)

| Name          | Affiliation                      | Country       | Title of the Paper                                                      | How Futurewei Supports the Author                                                                                     |
|---------------|----------------------------------|---------------|------------------------------------------------------------------------|-----------------------------------------------------------------------------------------------------------------------|
| Wei Jiang     | Futurewei Technologies Inc.     | USA           | Image and Video Compression using Generative Sparse Representation with Fidelity Controls | Futurewei provides resources and infrastructure to support the research and development of image and video compression methods.  |
| Wei Wang      | Futurewei Technologies Inc.     | USA           | Image and Video Compression using Generative Sparse Representation with Fidelity Controls | Futurewei supports the a

It turns out that the contents after rows of authors could have different format. The previous assumption that the last two rows are not part of the table is incorrect. there is a case in which the last 8 rows are not part of the author table. therefore, the code to remove the last two rows can't handle all cases. 
The solution is to only keep the rows of authors. and insert those after the auhor rows into a column called comment. 


In [48]:
# use pdf_file 2404.06706 as an example to test the code
content = completion.choices[0].message.content
# Find the start and end of the table
table_start = content.find('| Name')
table_end = content.find('**Summary of Futurewei\'s Support**')
table_markdown = content[table_start:table_end].strip()

# Split the markdown table into rows
rows = table_markdown.split('\n')

# print the 3rd rows
print('3rd rows:', rows[2])

# print the 4th rows    
print('4th rows:', rows[3])

# print the 5th rows    
print('5th rows:', rows[4])

# print the 6th rows    
print('6th rows:', rows[5])

# print the 7th rows    
print('7th rows:', rows[6])

# print the 8th rows    
print('8th rows:', rows[7])

# print the 9th rows    
print('9th rows:', rows[8])




3rd rows: | Wei Jiang     | Futurewei Technologies Inc.     | USA           | Image and Video Compression using Generative Sparse Representation with Fidelity Controls | Futurewei provides resources and infrastructure to support the research and development of image and video compression methods.  |
4th rows: | Wei Wang      | Futurewei Technologies Inc.     | USA           | Image and Video Compression using Generative Sparse Representation with Fidelity Controls | Futurewei supports the authors by offering technical guidance, collaboration opportunities, and access to relevant datasets.  |
5th rows: 
6th rows: ### Explanation of the Support Column
7th rows: - **Resources and Infrastructure**: Indicates that Futurewei provides the necessary tools and facilities to conduct the research.
8th rows: - **Technical Guidance**: Implies that Futurewei offers expertise to help refine and develop the proposed methods further.
9th rows: - **Collaboration Opportunities**: Suggests that there may 

In [52]:

# analyzing different response from gpt4, rows with authors are seperated from the left by 1 empty row. therefore, extract all rows before the empty row. 
# the empty row and the rows after the empty row should be added to a new column called comment. 

# Assuming 'rows' contains the data from table_markdown, locate the empty row
empty_row_index = rows.index('')
print('empty_row_index:', empty_row_index)

# Combine all subsequent rows into one sentence 
support_explanation = []
for row in rows[empty_row_index+1:]:        
        for explanation_row in rows[rows.index(row) + 1:]:  # Start from the next row
            if explanation_row.strip():  # Only add non-empty rows
                support_explanation.append(explanation_row.strip())
        break  # Exit the loop after processing the explanation

# Combine the explanation into one sentence
combined_explanation = ' '.join(support_explanation)
print("Combined Explanation:", combined_explanation)

# extract all rows after the format row-2nd row and before the empty row
for row in rows[2:empty_row_index]:  # Skip the header row and format row
        new_row = [None] * len(headers)  # Initialize a list with None values
        new_row[headers.index('Author')] = row.split('|')[1].strip()    # Name->Author
        new_row[headers.index('Affiliation')] = row.split('|')[2].strip()   # Affiliation
        new_row[headers.index('国家')] = row.split('|')[3].strip()     # Country->国家
        new_row[headers.index('Title of Publication\\')] = row.split('|')[4].strip()  # Title of Paper
        new_row[headers.index('资助方式')] = row.split('|')[5].strip() # Support
        print("New row:", new_row)
        # append the new row to the df
        df.loc[len(df)] = new_row

# obtain the content of '资助方式' column of the last row in df
last_row_support = df.iloc[-1]['资助方式']

# Append the combined_explanation to last_row_support  
df.loc[len(df)-1, '资助方式'] = last_row_support + combined_explanation

# print the last row of df
print('last row of df:', df.iloc[-1])


empty_row_index: 4
Combined Explanation: - **Resources and Infrastructure**: Indicates that Futurewei provides the necessary tools and facilities to conduct the research. - **Technical Guidance**: Implies that Futurewei offers expertise to help refine and develop the proposed methods further. - **Collaboration Opportunities**: Suggests that there may be chances to work with other experts in the field through Futurewei’s network. - **Access to Relevant Datasets**: Indicates that Futurewei may have the ability to provide datasets necessary for the training and evaluation of the proposed compression methods. Feel free to ask if you need any further modifications or additional information
New row: ['Wei Jiang', 'Futurewei Technologies Inc.', 'USA', 'Image and Video Compression using Generative Sparse Representation with Fidelity Controls', None, 'Futurewei provides resources and infrastructure to support the research and development of image and video compression methods.']
New row: ['Wei 

In [53]:
# since pdf_file 2404.06706 pass the test cases, clean up the code and run the rest of the pdf files    

# clean up df by removing rows with index 68 to 71
df = df.drop(range(68, 71))



In [83]:
# print rows with index 64 to the last row of df
print('rows with index 64 to the last row of df:', df.iloc[64:])

# remove all the rows i printed out, which is 64 to the last row        
df = df.drop(range(123, 124))

# drop the last row of df


# print the last row of df
print('last row of df:', df.iloc[-1])


rows with index 64 to the last row of df:        Author                  Affiliation   国家  \
123  Wei Wang  Futurewei Technologies Inc.  USA   

                                 Title of Publication\    来源  \
123  Image and Video Compression using Generative S...  None   

                                                  资助方式  
123  Futurewei supports the authors by offering tec...  
last row of df: Author                                                      Ibrahim Haddad
Affiliation                                               Linux Foundation
国家                                                  San Francisco, CA, USA
Title of Publication\    The Model Openness Framework: Promoting Comple...
来源                                                                    None
资助方式                     Futurewei actively engages with the Linux Foun...
Name: 63, dtype: object


In [79]:
#also drop the row with index 64, print it out and check if it's correct
print('pdf_file:', pdf_file)
text = extract_text_from_pdf(pdf_path)

prompt = f"step 1: extract the info for each author, including name, its affiliation, country, title of the paper, and how does futurewei support the author in the paper {text}; \
step 2, add a new row for each author in the table;" 

# Send the prompt to the OpenAI API and get the response
completion = client.chat.completions.create(
    model="gpt-4o-mini",  # Using "gpt-4" or another appropriate model
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", 
            "content": prompt
        }
    ]
)

print('completion:', completion.choices[0].message)

pdf_file: 2403.13784.pdf
completion: ChatCompletionMessage(content='Here’s the extracted information for each author from the paper titled "THE MODEL OPENNESS FRAMEWORK: PROMOTING COMPLETENESS AND OPENNESS FOR REPRODUCIBILITY, TRANSPARENCY, AND USABILITY IN ARTIFICIAL INTELLIGENCE":\n\n| Name                | Affiliation               | Country           | Title of the Paper                                                                 | Futurewei Support                            |\n|---------------------|--------------------------|--------------------|------------------------------------------------------------------------------------|----------------------------------------------|\n| Matt White          | Linux Foundation         | San Francisco, CA, USA | THE MODEL OPENNESS FRAMEWORK: PROMOTING COMPLETENESS AND OPENNESS FOR REPRODUCIBILITY, TRANSPARENCY, AND USABILITY IN ARTIFICIAL INTELLIGENCE | Active involvement in the development of the Model Openness Framework. |\n| Ibrahim

In [86]:
# proceed with the left over pdf files          

# Assuming 'rows' contains the data from table_markdown, locate the empty row
for pdf_file in pdf_files[4:]:
    pdf_path = 'arxiv_pdfs/' + pdf_file
    print('pdf_path:', pdf_path)
    text = extract_text_from_pdf(pdf_path)
    prompt = f"step 1: extract the info for each author, including name, its affiliation, country, title of the paper, and how does futurewei support the author in the paper {text}; \
    step 2, add a new row for each author in the table;" 

    # Send the prompt to the OpenAI API and get the response
    completion = client.chat.completions.create(
        model="gpt-4o-mini",  # Using "gpt-4" or another appropriate model
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", 
             "content": prompt
            }
        ]
    )

    # Extract the table part from the content
    content = completion.choices[0].message.content
    # Find the start and end of the table which contains necessary info
    table_start = content.find('| Name')
    table_end = content.find('**Summary of Futurewei\'s Support**')
    table_markdown = content[table_start:table_end].strip()

    # Split the markdown table into rows
    rows = table_markdown.split('\n')
    # print the table markdown and the shape of table_markdown
    print(table_markdown)

    # identify whether there is  the empty row which seperate the must-have info from the optional info
    if '' in rows:
        empty_row_index = rows.index('')
        print('empty_row_index:', empty_row_index)
        final_row_musthave = empty_row_index
        # Combine all subsequent rows (belonging to the optional info) into  one sentence 
        support_explanation = []
        for row in rows[empty_row_index+1:]:        
            for explanation_row in rows[rows.index(row) + 1:]:  # Start from the next row
                if explanation_row.strip():  # Only add non-empty rows
                    support_explanation.append(explanation_row.strip())
            break  # Exit the loop after processing the explanation
            # Combine the explanation into one sentence
        combined_explanation = ' '.join(support_explanation)
        print("Combined Explanation:", combined_explanation)
    else:
        final_row_musthave = len(rows)



    # extract all rows (must-have) after the format row-2nd row and before the empty row
    for row in rows[2:final_row_musthave]:  # Skip the header row and format row
            new_row = [None] * len(headers)  # Initialize a list with None values
            new_row[headers.index('Author')] = row.split('|')[1].strip()    # Name->Author
            new_row[headers.index('Affiliation')] = row.split('|')[2].strip()   # Affiliation
            new_row[headers.index('国家')] = row.split('|')[3].strip()     # Country->国家
            new_row[headers.index('Title of Publication\\')] = row.split('|')[4].strip()  # Title of Paper
            new_row[headers.index('资助方式')] = row.split('|')[5].strip() # Support
            print("New row:", new_row)
            # append the new row to the df
            df.loc[len(df)] = new_row

    # obtain the content of '资助方式' column of the last row in df
    last_row_support = df.iloc[-1]['资助方式']

    # Append the combined_explanation to last_row_support
    if '' in rows:
        df.loc[len(df)-1, '资助方式'] = last_row_support + combined_explanation

    # print the last row of df
    print('last row of df:', df.iloc[-1])

pdf_path: arxiv_pdfs/2403.16951.pdf
| Name           | Affiliation                                                       | Country            | Title of the Paper                                                                                 | How Futurewei Supports the Author                                      |
|----------------|------------------------------------------------------------------|--------------------|---------------------------------------------------------------------------------------------------|-----------------------------------------------------------------------|
| Reza Farahani  | Alpen-Adria-Universität Klagenfurt, Faculty of Technical Sciences | Austria             | "Network-Assisted Delivery of Adaptive Video Streaming Services through CDN, SDN, and MEC"       | Futurewei may provide insights and funding, enhancing research impact by enabling access to advanced networking technologies. The mention of support is implicit in the context of collaboration wi